In [105]:
# predict the next char
import tensorflow as tf
import numpy as np
import random
import pandas as pd
%matplotlib inline

In [113]:
def make_mini_batch(train_data, size_of_mini_batch, length_of_sequences):
    inputs  = np.empty(0)
    outputs = np.empty(0)
    for _ in range(size_of_mini_batch):
        index   = random.randint(0, len(train_data) - length_of_sequences - 1)
        inputs  = np.append(inputs, train_data[index:index + length_of_sequences])
        outputs = np.append(outputs, train_data[index + length_of_sequences + 1])
    inputs  = inputs.reshape(-1, length_of_sequences, 128)
    outputs = outputs.reshape(-1, 128)
    return (inputs, outputs)

In [114]:
def make_prediction_initial(train_data, index, length_of_sequences):
    return train_data[index:index + length_of_sequences], train_data[index + length_of_sequences + 1]
#inputs, _  = make_prediction_initial(train_data, 0, length_of_initial_sequences)
#inputs.shape

In [123]:
train_data_path             = "./char_rnn.ipynb"
num_of_char                 = 128
num_of_input_nodes          = 128
num_of_hidden_nodes         = 4
num_of_output_nodes         = 128
length_of_sequences         = 30
num_of_training_epochs      = 400
length_of_initial_sequences = 30
num_of_prediction_epochs    = 100
size_of_mini_batch          = 2
learning_rate               = 0.001
forget_bias                 = 1.0

In [124]:
#train_data = np.load(train_data_path)
f = open(train_data_path)
text = f.read()
f.close()
train_data = np.eye(num_of_char)[[ord(x) for x in text]]
print("{}".format(train_data[4:5]))

[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.]]


In [125]:
a, b = make_mini_batch(train_data, size_of_mini_batch, length_of_sequences)
a.shape
b.shape

(2, 128)

In [126]:
def inference(input_ph):
    with tf.name_scope("inference") as scope:
        weight1_var = tf.Variable(tf.truncated_normal([num_of_input_nodes, num_of_hidden_nodes], stddev=0.1), name="weight1")
        weight2_var = tf.Variable(tf.truncated_normal([num_of_hidden_nodes, num_of_output_nodes], stddev=0.1), name="weight2")
        bias1_var   = tf.Variable(tf.truncated_normal([num_of_hidden_nodes], stddev=0.1), name="bias1")
        bias2_var   = tf.Variable(tf.truncated_normal([num_of_output_nodes], stddev=0.1), name="bias2")

        weight1_hist = tf.histogram_summary("layer1/weights", weight1_var)
        weight2_hist = tf.histogram_summary("layer2/weights", weight2_var)
        bias1_hist = tf.histogram_summary("layer1/biases", bias1_var)
        bias2_hist = tf.histogram_summary("layer2/biases", bias2_var)
        
        # pre rnn
        in1 = tf.transpose(input_ph, [1, 0, 2])         # (batch, sequence, data) -> (sequence, batch, data)
        in2 = tf.reshape(in1, [-1, num_of_input_nodes]) # (sequence, batch, data) -> (sequence * batch, data)
        in3 = tf.matmul(in2, weight1_var) + bias1_var
        in4 = tf.split(0, length_of_sequences, in3)     # sequence * (batch, data)

        _cell = tf.nn.rnn_cell.BasicLSTMCell(num_of_hidden_nodes, forget_bias=forget_bias, state_is_tuple=True)
        cell = tf.nn.rnn_cell.MultiRNNCell([_cell] * 1, state_is_tuple=True)
        rnn_output, states_op = tf.nn.rnn(cell, in4, dtype=tf.float32)
        
        # post rnn
        #output_op = tf.matmul(rnn_output[-1], weight2_var) + bias2_var
        out1 = tf.matmul(rnn_output[-1], weight2_var) + bias2_var
        output_op = out1
        #output_argmax = tf.argmax(out1, 0)

    return output_op

In [127]:
def loss(output_op):
    with tf.name_scope("loss") as scope:
        cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(output_op, supervisor_ph))
        loss_op = cross_entropy
        tf.scalar_summary("loss", loss_op)
    return loss_op

In [128]:
def train(loss_op):
    with tf.name_scope("training") as scope:
        #optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
        #optimizer = tf.train.AdagradOptimizer(learning_rate=learning_rate)
        optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
        training_op = optimizer.minimize(loss_op)
    return training_op

In [129]:
with tf.Graph().as_default():
    input_ph      = tf.placeholder(tf.float32, [None, length_of_sequences, num_of_input_nodes], name="input")
    supervisor_ph = tf.placeholder(tf.float32, [None, num_of_output_nodes], name="supervisor")

    output_op = inference(input_ph)
    loss_op = loss(output_op)
    training_op = train(loss_op)

    summary_op = tf.merge_all_summaries()
    init = tf.initialize_all_variables()
    #istate_ph = cell.zero_state(size_of_mini_batch, tf.float32)

    with tf.Session() as sess:
        saver = tf.train.Saver()
        summary_writer = tf.train.SummaryWriter("data", graph=sess.graph)
        # random seed fix
        random.seed(0)
        np.random.seed(0)
        tf.set_random_seed(0)
        # init variables
        sess.run(init)

        for epoch in range(num_of_training_epochs):
            inputs, supervisors = make_mini_batch(train_data, size_of_mini_batch, length_of_sequences)
            #state = sess.run(istate_ph)

            train_dict = {
                input_ph:      inputs,
                supervisor_ph: supervisors,
            }
            sess.run(training_op, feed_dict=train_dict)

            if (epoch + 1) % 10 == 0:
                summary_str, train_loss = sess.run([summary_op, loss_op], feed_dict=train_dict)
                summary_writer.add_summary(summary_str, epoch)
                print("train#%d, train loss: %e" % (epoch + 1, train_loss))

        inputs, _  = make_prediction_initial(train_data, 0, length_of_initial_sequences)
        outputs = np.empty(0)

#        istate_ph = cell.zero_state(1, tf.float32)
#        sess.run(istate_ph)

        print("prediction: ")
        for epoch in range(num_of_prediction_epochs):
            pred_dict = {
                input_ph:  [inputs]
            }
#            output, states = sess.run([output_op, states_op], feed_dict=pred_dict)
            output = sess.run(output_op, feed_dict=pred_dict)
            inputs  = np.delete(inputs, 0, 0)
            inputs  = np.append(inputs, output, 0)
            outputs = np.append(outputs, output)
        
        output_ascii = np.argmax(outputs.reshape(num_of_prediction_epochs, num_of_output_nodes), axis=1)
        print("asciis: ", output_ascii)
        print("outputs:", [chr(x) for x in output_ascii])
        #saver.save(sess, "data/model")

train#10, train loss: 4.917773e+00
train#20, train loss: 4.894240e+00
train#30, train loss: 4.726043e+00
train#40, train loss: 4.785027e+00
train#50, train loss: 4.770592e+00
train#60, train loss: 4.798577e+00
train#70, train loss: 4.715875e+00
train#80, train loss: 4.741944e+00
train#90, train loss: 4.820520e+00
train#100, train loss: 4.589970e+00
train#110, train loss: 4.663841e+00
train#120, train loss: 4.580944e+00
train#130, train loss: 4.397003e+00
train#140, train loss: 4.536016e+00
train#150, train loss: 4.179140e+00
train#160, train loss: 4.017682e+00
train#170, train loss: 4.119983e+00
train#180, train loss: 4.374049e+00
train#190, train loss: 4.143290e+00
train#200, train loss: 3.521620e+00
train#210, train loss: 3.423774e+00
train#220, train loss: 4.603148e+00
train#230, train loss: 3.692237e+00
train#240, train loss: 3.961233e+00
train#250, train loss: 3.819065e+00
train#260, train loss: 3.976480e+00
train#270, train loss: 3.084389e+00
train#280, train loss: 4.147577e+00
t

In [130]:
np.argmax(outputs.reshape(5, 128), axis=1)

ValueError: total size of new array must be unchanged